In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
with open("contacts.json", "r") as content:
    data = json.load(content)
df = pd.DataFrame(data)

In [3]:
class DisjSet:
    def __init__(self, n):
        self.rank = [1] * n
        self.parent = [i for i in range(n)]
    
    def find(self,x):
        if self.parent[x] != x:
            self.parent[x]= self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        xset = self.find(x)
        yset = self.find(y)
        if xset == yset:
            return
        if self.rank[xset] < self.rank[yset]:
            self.parent[xset] = yset
        elif self.rank[xset] > self.rank[yset]:
            self.parent[yset] = xset
        else:
            self.parent[yset] = xset
            self.rank[xset] = self.rank[xset] + 1

In [4]:
# construct a disjoint set
disjoin_set = DisjSet(df.shape[0] * 3)
dic = {}
dic_key = {}
for row in range(df.shape[0]):
    id_, email, phone, contacts, order = df.iloc[row]
    n_email, n_phone, n_order = 0, 0, 0
    if email != '': 
        dic[email] = dic.get(email, id_*3)      
        n_email = dic[email]
        dic_key[n_email] = email
        
    if phone != '': 
        dic[phone] = dic.get(phone, id_*3+1)
        n_phone = dic[phone]
        dic_key[n_phone] = phone
    if order != '':
        dic[order] = dic.get(order, id_*3+2)
        n_order = dic[order]
        dic_key[n_order] = order
        
    if n_email != 0 and n_phone != 0:
        disjoin_set.union(n_email, n_phone)
    if n_email != 0 and n_order != 0:
        disjoin_set.union(n_email, n_order)
    if n_phone != 0 and n_order != 0:
        disjoin_set.union(n_phone, n_order)    

In [5]:
lst = [0] * df.shape[0] * 3
for i in dic.keys():
    lst[dic[i]] = disjoin_set.find(dic[i])

In [6]:
new_dic = {}
for row in range(df.shape[0]):
    id_, email, phone, contacts, order = df.iloc[row]
    n = 0
    if email != '': 
        n = lst[dic[email]]
    if phone != '':
        n = lst[dic[phone]]
    if order != '':
        n = lst[dic[order]]
    if n != 0:
        new_dic[n] = new_dic.get(n,set())
        new_dic[n].add(id_)
    else:
        new_dic[id_] = set()
        new_dic[id_].add(id_)

In [7]:
lst_final = [0] * df.shape[0]
for key,value in new_dic.items():
    my_sum = 0
    final = list(value)
    final.sort()
    for i in value:    
        my_sum += df.loc[i]['Contacts']
    final_str = '-'.join(str(j) for j in final) + ', ' + str(my_sum)
    for i in value:
        lst_final[i] = final_str

In [8]:
submission = pd.DataFrame(lst_final)
submission = submission.reset_index()
submission.columns = ['ticket_id','ticket_trace/contact']
submission.to_csv('submission.csv')

In [9]:
submission

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
499995,499995,"499995, 2"
499996,499996,"499996, 4"
499997,499997,"499997, 2"
499998,499998,"121111-499998, 5"
